<a target="_blank" href="https://colab.research.google.com/github/pr-Mais/arabic-reverse-dictionary/blob/main/code/mt5_training_shared_task.ipynb">
  <img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/>
</a>

# Fine-tuning MT5 for Reverse Dictionary

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## Requirements

In [ ]:
!pip3 install -q -U datasets==2.18.0
!pip3 install -q -U transformers==4.39.2
!pip3 install -q -U evaluate==0.4.1
!pip3 install -q -U scikit-learn==1.4.1.post1

In [ ]:
import pandas as pd
from tqdm import tqdm

from datasets import Dataset, DatasetDict

import torch
import torch.nn as nn
import torch.nn.functional as F

from transformers import (
    AutoConfig,
    AutoTokenizer,
    AutoModelForSeq2SeqLM,
    AdamW,
    set_seed,
    get_linear_schedule_with_warmup,
)
from transformers.modeling_outputs import Seq2SeqLMOutput

# For reproducibility
set_seed(123)
# Define needed parameters
checkpoint = "UBC-NLP/AraT5v2-base-1024"
epochs = 3
lr = 5e-4
# The max length of the embeddings, this should match the length of the shared task target embeddings
max_length = 256
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Loading and preprocessing data

In [ ]:
train_df = pd.read_json(
    "../data/shared-task/train_with_examples.json", encoding="utf-8"
)
val_df = pd.read_json("../data/shared-task/dev.json", encoding="utf-8")
test_df = pd.read_json("../data/shared-task/test.json", encoding="utf-8")

In [ ]:
train_df.head()

In [ ]:
# Merge train and val into one dict with keys train and val
train_val_dict = {
    "train": train_df.to_dict("records"),
    "val": val_df.to_dict("records"),
}

In [ ]:
# Convert to HF dataset
train_ds = Dataset.from_pandas(train_df, split="train")
val_ds = Dataset.from_pandas(val_df, split="validation")
test_ds = Dataset.from_pandas(test_df, split="test")
dataset = DatasetDict({"train": train_ds, "val": val_ds})

In [ ]:
# Take a look at the dataset
dataset

In [ ]:
# Tokenization step
tokenizer = AutoTokenizer.from_pretrained(checkpoint, legacy=False)

padding = "max_length"
max_input_length = 256


def preprocess_function(items):
    # The inputs are the glosses + examples
    if "examples" in items:
        glosses = [
            f"{gloss}. {example[0]}، {example[1]}"
            for gloss, example in zip(items["gloss"], items["examples"])
        ]
    else:
        glosses = items["gloss"]

    model_inputs = tokenizer(
        glosses,
        max_length=max_input_length,
        padding=padding,
        truncation=True,
        return_tensors="pt"
    )

    # Adding the 3 types of target embeddings, if they are available.
    if "electra" in items:
        model_inputs["electra"] = items["electra"]  # Electra embeddings
    if "bertseg" in items:
        model_inputs["bertseg"] = items["bertseg"]  # BERTseg embeddings
    if "bertmsa" in items:
        model_inputs["bertmsa"] = items["bertmsa"]  # BERTmsa embeddings

    targets = [ex for ex in items["word"]]
    # encode the words
    labels = tokenizer(
        targets, max_length=max_length, padding=padding, truncation=True, return_tensors="pt"
    ).input_ids

    # important: we need to replace the index of the padding tokens by -100
    # such that they are not taken into account by the CrossEntropyLoss
    labels_with_ignore_index = []
    for labels_example in labels:
        labels_example = [label if label != 0 else -100 for label in labels_example]
        labels_with_ignore_index.append(labels_example)

    model_inputs["labels"] = labels_with_ignore_index

    return model_inputs


# Final mapping of the dataset
dataset = dataset.map(preprocess_function, batched=True)

In [ ]:
from torch.utils.data import DataLoader

dataset.set_format(
    type="torch",
    columns=["input_ids", "attention_mask", "labels", "electra", "bertseg", "bertmsa"],
    output_all_columns=False,
)
train_dataloader = DataLoader(dataset["train"], shuffle=True, batch_size=8)
valid_dataloader = DataLoader(dataset["val"], batch_size=4)

In [ ]:
print(
    f"Electra embeddings shape: 1, {dataset['train']['electra'].shape[1]}\n"
    f"BERTseg embeddings shape: 1, {dataset['train']['bertseg'].shape[1]}\n"
    f"BERTmsa embeddings shape: 1, {dataset['train']['bertmsa'].shape[1]}\n"
)

## Training

In [ ]:
from typing import Literal
import torch.nn as nn

# Loss functions
mse_loss = nn.MSELoss()

TargetEmbeddingType = Literal["electra", "bertseg", "bertmsa"]


def train(
    dataloader,
    optimizer_,
    scheduler_,
    device_,
    target: TargetEmbeddingType = "electra",
    validate=False,
):
    # Use global variable for model.
    global model
    # Tracking variables.
    predictions = []
    ground_truth = []
    # Total loss for this epoch.
    total_loss = 0
    if not validate:
        model.train()
    if validate:
        model.eval()

    # For each batch of training data...
    for batch in tqdm(dataloader):
        ground_truth += batch[target].numpy().tolist()
        inputs = {
            k: v.to(device_)
            for k, v in batch.items()
            if k in ["input_ids", "attention_mask"]
        }
        labels = batch["labels"].to(device)
        if not validate:
            model.zero_grad()
        embeddings = model(**inputs, labels=labels, return_dict=True)
        # Loss is calculated on target embeddings, outside the model.
        loss = mse_loss(embeddings, batch[target].to(device_))
        total_loss += loss.item()
        if not validate:
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer_.step()
            scheduler_.step()
        predictions += embeddings.tolist()

    # Calculate the average loss over the training data.
    avg_epoch_loss = total_loss / len(dataloader)

    # Return all true labels and prediction for future evaluations.
    return ground_truth, predictions, avg_epoch_loss

In [ ]:
class RevDictModel(nn.Module):
    def __init__(self, max_len: int, checkpoint: str):
        super().__init__()
        # if from_pretrained:
        #     self.base_model = AutoModelForSeq2SeqLM.from_pretrained(
        #         checkpoint
        #     )  # Using AutoModel which is better for embedding extraction
        # else:
        model_config = AutoConfig.from_pretrained(checkpoint)
        self.base_model = AutoModelForSeq2SeqLM.from_config(model_config)

        # Redefining the linear layer to match the target embedding size (max_len)
        self.linear = nn.Linear(self.base_model.config.hidden_size, max_len)

    def forward(self, input_ids, attention_mask, **kwargs):
        # Only using the encoder part to generate embeddings
        outputs = self.base_model.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            return_dict=True,
        )
        pooled_emb = (
            outputs.last_hidden_state * attention_mask.unsqueeze(2)
        ).sum(dim=1) / attention_mask.sum(dim=1).unsqueeze(1)
        embedding = self.linear(pooled_emb)
        return embedding

    def save(self, file):
        torch.save(self, file)

    @staticmethod
    def load(file):
        return torch.load(file, map_location=device)

In [ ]:
model = RevDictModel(max_len=max_length, checkpoint=checkpoint)
model.to(device)

In [ ]:
optimizer = AdamW(
    model.parameters(),
    lr=2e-5,
    eps=1e-8,
)
# Total number of training steps is number of batches * number of epochs.
# `train_dataloader` contains batched data so `len(train_dataloader)` gives
# us the number of batches.
total_steps = len(train_dataloader) * epochs

# Create the learning rate scheduler.
scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=total_steps,
)

# Store the average loss after each epoch so we can plot them.
all_loss = {"train_loss": [], "val_loss": []}

# Loop through each epoch.

for epoch in range(epochs):
    print(f"Epoch {epoch}")
    train_labels, train_predict, train_loss = train(
        train_dataloader, optimizer, scheduler, device
    )
    valid_labels, valid_predict, val_loss = train(
        valid_dataloader, optimizer, scheduler, device, validate=True
    )

    # Print loss and accuracy values to see how training evolves.
    print("  train_loss: %.5f - val_loss: %.5f" % (train_loss, val_loss))
    print()

    # Store the loss value for plotting the learning curve.
    all_loss["train_loss"].append(train_loss)
    all_loss["val_loss"].append(val_loss)

# Plot loss curves.
# plot_dict(all_loss, use_xlabel="Epochs", use_ylabel="Value", use_linestyles=["-", "--"])

In [ ]:
save_directory = "/content/drive/MyDrive/mt5_shared_task_checkpoint_0"
model.save(save_directory)

## Evaluation

In [ ]:
# re-load the model
model = RevDictModel.load("../checkpoints/mt5_shared_task_checkpoint_0")

In [ ]:
import evaluate


def run_evaluate(sample):
    model.eval()
    outputs = model(
        input_ids=sample["input_ids"].to(device),
        attention_mask=sample["attention_mask"].to(device)
    )
    return outputs

predictions = []
for sample in tqdm(valid_dataloader):
    p = run_evaluate(sample)
    predictions += p

In [ ]:
len(predictions)

In [ ]:
predictions

### P@3 with Cosine Similarity

Here we use the out embeddings and find the top 3 similar words from the test set then find how many of them match the ground truth.

In [ ]:
from torch.nn.functional import cosine_similarity
from typing import List

model.eval()

# Calculate top n words similar to the output embedding
def get_top_n(emb: torch.Tensor, predictions: List[torch.Tensor], n: int = 5):
    scores = []
    for item in predictions:
        # Find the similarity score
        score = cosine_similarity(emb.to(device), item, dim=0)
        # Append to total results
        scores.append(score.item())
    # get top n
    return sorted(scores, reverse=True)[:n]

In [ ]:
scores = []
i = 0
for item in valid_dataloader.dataset:
    emb = item['electra']
    scores.append(get_top_n(emb, predictions, n=3))
    i += 1
    if i == 5:
        break

In [ ]:
# Get the average score
average_at_3 = [sum(x) / len(x) for x in zip(*scores)]
average = sum(average_at_3) / len(average_at_3)
average

# Use it